In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
from keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential

2023-06-11 13:51:54.537392: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 13:51:54.735831: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-11 13:51:54.738244: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 13:51:56.192794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_files(directory: str):
    return np.stack(list(map(lambda path: np.load(path), Path(directory).glob('**/*.npy'))))

X_train = load_files('../data/images_train')
y_train = load_files('../data/masks_train').squeeze()  # squeeze removes the dimension of size 1

# Channel as last dimension
X_train = np.swapaxes(X_train,1,3)

# Normalize the images
# Original shape of images, e.g. (40, 1024, 1024, 10)
original_shape = X_train.shape
# Reshape the images to 2D array, (40*1024*1024, 10)
X_train_2d = np.reshape(X_train, (-1, original_shape[-1]))
# Initialize and fit the MinMaxScaler
scaler = MinMaxScaler()
X_train_2d_scaled = scaler.fit_transform(X_train_2d)
# Reshape the images back to their original shape
X_train_scaled = np.reshape(X_train_2d_scaled, original_shape)

In [3]:
model = Sequential([
    InputLayer(input_shape=[1024, 1024, 10]),
    Conv2D(filters=1, kernel_size=5, strides=1, padding='same', activation='relu'),
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1024, 1024, 1)     251       
                                                                 
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [11]:
class MaskedLoss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        # If y_true is zero, the loss is zero.
        return tf.where(y_true == 0, 0.0, tf.keras.losses.mean_absolute_error(y_true - y_pred))

In [12]:
model.compile(loss=MaskedLoss, optimizer="adam", metrics=[MaskedLoss])

In [13]:
history = model.fit(X_train_scaled, 
                    y_train, 
                    epochs=10, 
                    batch_size=1, 
                    validation_split=0.1)